In [2]:
from gurobipy import *
import csv
import random
import multidict

In [16]:
# def input_generator():
#     # INPUT
#     age = random.randint(1, 179)
#     risk = random.randint(0, 50)
#     size = random.randint(5000, 9999)
#     # return INPUT
#     return float(risk)
#     # return [float(age), float(risk), float(size)]


# def output_generator():
#     # OUTPUT
#     sale = random.randint(500000000, 1000000000)

#     # return OUTPUT
#     return float(sale)
# input_generator(), output_generator()

In [17]:
# ## Data generator

# K = 3
# X = {}
# Y = {}
# DMU = []

# # number of DMU
# N = 5

# for j in range(N):
#     DMU.append(j)
#     X[j] = {}
#     Y[j] = output_generator()
#     for k in range(K):
#         X[j][k] = input_generator()
       

# DMU, X, Y

## Equation (1)

In [3]:
X = {0: {0: 1, 1: 1, 2: 1},
     1: {0: 2, 1: 2, 2: 2},
     2: {0: 3, 1: 3, 2: 3},
     3: {0: 4, 1: 4, 2: 4},
     4: {0: 5, 1: 5, 2: 5}}

Y = {0: 100,
     1: 100,
     2: 100,
     3: 100,
     4: 100}

DMU = [0, 1, 2, 3, 4]

In [5]:
try:
    K = 3
    H = K

    # number of DMU
    N = 5

    # 暫定
    e = 1

    m_k = [1, 1, 1]
    r_k = [1, 1, 1]

    w = [1/3, 1/3, 1/3]

    E = {}  # Efficiency

    m = Model("model")

    # add variables
    λ   = m.addVars(N, K, lb=0, vtype=GRB.CONTINUOUS, name="λ_") # λ_jk
    w_j = m.addVars(N, K, lb=0, vtype=GRB.CONTINUOUS, name="w_j_")

    s_neg = m.addVars(N, K, lb=0, vtype=GRB.CONTINUOUS, name="s_neg_")
    s_pos = m.addVars(N, K, lb=0, vtype=GRB.CONTINUOUS, name="s_pos_")

    x_o = m.addVars(K, vtype=GRB.CONTINUOUS, name="x_o_")
    y_o = m.addVars(K, vtype=GRB.CONTINUOUS, name="y_o_")

    y = m.addVars(N, K, vtype=GRB.CONTINUOUS, name="_")

    z = m.addVars(N, K, H, vtype=GRB.CONTINUOUS, name="_")

    m.update()    

    #  set Objective
    m.setObjective(sum(w[k] * (1-(1/m_k[k])*(sum(s_neg[i, k]/X[i][k] for i in range(m_k[k])))) for k in range(K)), GRB.MINIMIZE)

    ### X[i][k] -> X[i][k] 

   # add Constrs
    for k in range(K):
        m.addConstr((x_o[k] == sum(X[j][k] * λ[j, k] + s_neg[j, k] for j in range(N))), "C1")
        m.addConstr((y_o[k] == sum(y[j, k] * λ[j, k] - s_pos[j, k] for j in range(N))), "C2")

        m.addConstr(sum(e * λ[j, k] for j in range(N)) == 1, "C3")
        m.addConstr(sum(w_j[j, k] for j in range(N)) == 1, "C4")


    # --- K stage: 0, 1, 2 --- #
    # y_0 = z_1 = x_1
    # y_1 = z_2 = x_2

    m.addConstrs(y[j, 0] == z[j, 0, 1] for j in range(N))
    m.addConstrs(y[j, 1] == z[j, 1, 2] for j in range(N))
    m.addConstrs(y[j, 2] == Y[j] for j in range(N))


    # (k, h) pairs
    m.addConstr((sum(z[j ,0, 1] * λ[j, 0] for j in range(N)) == sum(z[j, 0, 1] * λ[j, 1] for j in range(N))), "C5-1")
    m.addConstr((sum(z[j ,1, 2] * λ[j, 1] for j in range(N)) == sum(z[j, 1, 2] * λ[j, 2] for j in range(N))), "C5-2")

    m.optimize()

except GurobiError:
    print('GurobiError reported')

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 126 columns and 88 nonzeros
Model fingerprint: 0xe4d08fde
Model has 5 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e-01, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 11 rows and 65 columns
Presolve time: 0.02s

Barrier solved model in 0 iterations and 0.02 seconds (0.00 work units)
Model is infeasible or unbounded


## Equation (2)
### variable returns-to-scale DEA model (BCC) 

In [8]:
I = 1
O = 1

epsilon = 0.001

In [10]:
X = {0: {0: 1, 1: 1, 2: 1},
     1: {0: 2, 1: 2, 2: 2},
     2: {0: 3, 1: 3, 2: 3},
     3: {0: 4, 1: 4, 2: 4},
     4: {0: 5, 1: 5, 2: 5}}

Y = {0: 100,
     1: 100,
     2: 100,
     3: 100,
     4: 100}

DMU = [0, 1, 2, 3, 4]

In [9]:

for r in DMU:
    try:
        # The decision variables
        theta, λ, Sr, Si = {}, {}, {}, {}

        # Initialize LP model
        m = Model("Dual_of_CRS_model")
        m.setParam('OutputFlag', False)  # Muting the optimize function

        # Add decision variables
        for k in DMU:
            λ[k] = m.addVar(vtype=GRB.CONTINUOUS, name="λ_%s" % k)
            theta[r] = m.addVar(vtype=GRB.CONTINUOUS, lb=-1000, name="theta_%s" % r)
            Sr[r] = m.addVars(O, lb=0, vtype=GRB.CONTINUOUS, name="Sr_%s" % r)
            Si[r] = m.addVars(I, lb=0, vtype=GRB.CONTINUOUS, name="Si_%s" % r)

        m.update()

        # Add objective function
        m.setObjective(theta[r]-epsilon * (sum(Si[i] for i in range(I))), GRB.MINIMIZE)

        # Add constraints

        # for each input
        for i in range(I):
            m.addConstr(quicksum(λ[k]*X[k][i] for k in DMU) + Si[r][i] == theta[r]*X[r][i])
        # for each output
        for j in range(O):
            m.addConstr(quicksum(λ[k]*Y[k][j] for k in DMU) - Sr[r][j]>= Y[r][j])

        m.addConstr(sum(λ[k] for k in DMU) == 1)
        
        # Start optimize the formulation
        m.optimize()

        # Store the result
        E[r] = "The efficiency of DMU %s:%0.3f" % (r, m.objVal)
    
    except GurobiError:
        print('GurobiError reported')

    # Print efficiency
    print(E[r])

TypeError: unsupported operand type(s) for +: 'int' and 'tupledict'